In [1]:
import networkx as nx
from itertools import chain
from collections import Counter
import ast
import matplotlib.pyplot as plt

In [2]:
draw_options = {
    'node_color': 'blue', 'node_size': 100, 'width': 3,
    'arrowstyle': '-|>', 'arrowsize': 12,
}

In [3]:
g = nx.readwrite.graphml.read_graphml('./full_20180709_gml')

In [4]:
for i in g.edges():
    if len(i) != 2:
        print(i)
        continue
    if type(i[0]) != str or type(i[1]) != str:
        print(i)

In [5]:
def parallelEdge(g):
    eSize = []
    countEdge = 0
    for i in g.edges():
        c = len(g[i[0]][i[1]])
        countEdge += c
        eSize.append(c)
    return Counter(eSize), countEdge
multiEdgeC, totalEdge = parallelEdge(g)
print(multiEdgeC)
print(totalEdge)

Counter({1: 55216, 2: 1098, 3: 27})
57493


In [6]:
gSimple = nx.DiGraph(g.to_directed())

In [7]:
print(type(g))
print(type(gSimple))
print("Edge: " + str(g.number_of_edges()))
print("Node: " + str(g.number_of_nodes()))
print("Edge: " + str(gSimple.number_of_edges()))
print("Node: " + str(gSimple.number_of_nodes()))

<class 'networkx.classes.multidigraph.MultiDiGraph'>
<class 'networkx.classes.digraph.DiGraph'>
Edge: 56341
Node: 20930
Edge: 55774
Node: 20930


> Get useable attributes

In [8]:
def getNodeAttrib(g):
    return set(chain(*[(g.node[n].keys()) for n in g.nodes()]))
def getEdgeAttrib(g):
    return set(chain(*[(g.edges[i[0],i[1],0].keys()) for i in g.edges()]))

In [9]:
print("Node: " , getNodeAttrib(g))
print("Edge: " , getEdgeAttrib(g))

Node:  {'osmid', 'ref', 'lat', 'highway', 'lon', 'x', 'y'}
Edge:  {'access', 'tunnel', 'osmid', 'bridge', 'oneway', 'name', 'ref', 'junction', 'width', 'geometry', 'highway', 'service', 'lanes', 'maxspeed', 'length'}


> Extract edges with attribute 'bridge' is true

In [10]:
def extractBridges(g):
    bridge_nodes = []
    for e in g.edges():
        from_ = e[0]
        to_ = e[1]
        test_edge = g[from_][to_]
        if 'bridge' in test_edge and test_edge['bridge'] == 'yes':
            bridge_nodes.append(from_)
            bridge_nodes.append(to_)
    return g.subgraph(bridge_nodes).copy()

g_bridge = extractBridges(gSimple)

In [11]:
print(type(g_bridge))
print("Edge: " + str(g_bridge.number_of_edges()))
print("Node: " + str(g_bridge.number_of_nodes()))

<class 'networkx.classes.digraph.DiGraph'>
Edge: 870
Node: 694


> Load csv bridge markings

In [12]:
def read_marking_csv(fp):
    marks = []
    with open(fp, 'r') as f:
        for l in f.read().splitlines():
            four_osmid = l.replace(' ','').split(',')
            if len(four_osmid) != 4:
                raise NotImplementedError
            marks.append(four_osmid)
    return marks
marks = read_marking_csv('./a.csv')

In [13]:
marks

[['683700398', '104199519', '683700442', '683700447'],
 ['271201214', '5312910301', '5312910301', '1460700301'],
 ['339260059', '339258687', '686444866', '3227655838'],
 ['688524794', '688525038', '688525044', '688524822'],
 ['4575931118', '687157694', '1370793639', '4575931040'],
 ['687153341', '687154275', '687154277', '687153353'],
 ['684861957', '684844042', '684844055', '684861842'],
 ['684996191', '271191647', '684996280', '684996205']]

> Find names for all 'bridges'

In [14]:
# Unique bridge names
def findUniqueNames(g):
    tmpAll = {}
    for e in g.edges():
        test_edge = g[e[0]][e[1]]
        # Default 'noname' routine
        tmp_name = '[Unnamed]'
        if 'name' in test_edge:
            tmp_name = test_edge['name']
            if '[' and ']' in tmp_name:
                # Parse list and sort to guarantee order
                tmp_name = ast.literal_eval(tmp_name).sort()
        dictKeyName = str(tmp_name)
        if dictKeyName not in tmpAll:
            tmpAll[dictKeyName] = []
        tmpAll[dictKeyName].append((e[0],e[1]))
    return tmpAll, tmpAll.keys()
allEdgeByName, uniqueNames = findUniqueNames(g_bridge)
uniqueNames

dict_keys(['Centre Avenue', 'Chatham Street', '[Unnamed]', 'Highland Park Bridge', 'Lowrie Street', 'Greentree Road', 'Penn-Lincoln Parkway East', 'South Negley Avenue', 'Woodville Avenue', 'Minnotte Square', 'Edgebrook Avenue', 'Saw Mill Run Boulevard', 'Smithfield Street', 'None', 'Idlewood Avenue', 'Bigelow Boulevard', 'Veterans Bridge', 'South Aiken Avenue', 'Crosstown Boulevard', 'East 5th Avenue', 'McKnight Road', 'East Prospect Avenue', 'Boulevard of the Allies', 'Penn-Lincoln Parkway West', 'Chestnut Street', 'California Avenue', 'Forward Avenue', 'Isabella Street', 'Ohio River Boulevard', 'East Busway', 'Bell Avenue', 'Allegheny Valley Expressway', 'Timberland Avenue', 'Parkway North', 'East Street', 'Colerain Street', 'North Shore Drive', 'Library Road', 'Restricted Lane', 'South Highland Avenue', 'Spirit Street', 'Forbes Avenue', '31st Street Bridge', 'Liberty Avenue', 'Herron Avenue', 'Ansonia Place', 'West Elizabeth Street', 'Stanhope Street', 'Edgewood Avenue', 'South Mil

In [15]:
tmpC = {}
for k, v in allEdgeByName.items():
    if type(v) is list:
        tmpC[k] = len(v)
    else:
        print(v)
# Counter(tmpC)

In [16]:
def addPseudoEdge(g, uid, from_1, to_1, from_2, to_2):
    nodeName = "pseudo_" + str(uid)
    oriEdgeLeft = g[from_1][to_1]
    oriEdgeRight = g[from_2][to_2]
    newLeftName = 'Unnamed'
    newRightName = 'Unnamed'
    if 'name' in oriEdgeLeft:
        newLeftName = oriEdgeLeft['name']
    if 'name' in oriEdgeRight:
        newRightName = oriEdgeRight['name']
    g.add_node(nodeName, note="Fake")
    # Notice the direction
    g.add_edge(from_1, nodeName, name = (newLeftName + "_p1"))
    g.add_edge(nodeName, to_1, name = (newLeftName + "_p2"))
    g.add_edge(from_2, nodeName, name = (newRightName + "_p1"))
    g.add_edge(nodeName, to_2, name = (newRightName + "_p2"))
    # Remove old edge
    g.remove_edge(from_1, to_1)
    g.remove_edge(from_2, to_2)

> Try add nodes & edges

In [17]:
pseudo_count = 0
for m in marks:
    uid = "test" + str(pseudo_count)
    addPseudoEdge(gSimple, uid, m[0], m[1], m[2], m[3])    
    pseudo_count += 1

In [18]:
nx.readwrite.graphml.write_graphml(gSimple, "full_20180709_simpilfied_gml")

---
> Below are useless code

In [ ]:
c = 0
for e in g.edges(data=all):
    c+=1
    print("%s,%s" % (e[0],e[1]))
    if c>2:
        break


In [ ]:
g['683704708']['275755069'][0]

In [ ]:
s = g['104185739']['105697570']
for i in s:
    print(s[i])

In [ ]:
g.nodes['104551796']

In [ ]:
#nx.draw_networkx(g, arrows=True, **draw_options)

---
> Below are proto-code

In [ ]:
allEdgeByName['Veterans Bridge']

In [ ]:
tmp_g = g_bridge.subgraph(['684861957', '684844042','684907711', '684861957','684861842', '684907553','684844055', '684861842']).copy()

In [ ]:
fig = plt.figure()
plt.axis('off')
nx.draw_networkx(tmp_g, arrows=True, **draw_options)

In [ ]:
for e in tmp_g.edges():
    from_ = e[0]
    to_ = e[1]
    print("%s,%s" % (from_,to_))

In [ ]:
addPseudoEdge(tmp_g, 1, '684907711', '684861957', '684844055', '684861842')
addPseudoEdge(tmp_g, 2, '684861957', '684844042', '684861842', '684907553')

In [ ]:
fig = plt.figure()
plt.axis('off')
nx.draw_networkx(tmp_g, arrows=True)

In [ ]:
for e in tmp_g.edges():
    from_ = e[0]
    to_ = e[1]
    print("%s,%s" % (from_,to_))